# Anomaly detection with Linear Regression
In this laboratory, we will use a Linear Regression to predict the average Inter Arrival Time (IAT) of packets and to spot anomalies. In particular, we make our model learning the average IAT of benign flows and we predict the IAT of new traffic. If the squared error between the prediction and the actual value is higher than a threshold, we label the sample (flow) as an anomaly. After that, we compute the accuracy scores.
We will train a linear regression model on a dataset of benign traffic, and then we will test the trained model with DDoS attack traffic.

We will use a dataset of benign and various DDoS attacks from the CIC-DDoS2019 dataset (https://www.unb.ca/cic/datasets/ddos-2019.html).
The network traffic has been previously pre-processed in a way that packets are grouped in bi-directional traffic flows using the 5-tuple (source IP, destination IP, source Port, destination Port, protocol). Each flow is represented with 21 packet-header features computed from max 1000 packets:

| Feature nr.         | Feature Name |
|---------------------|---------------------|
| 00 | timestamp (mean IAT) | 
| 01 | packet_length (mean)| 
| 02 | IP_flags_df (sum) |
| 03 | IP_flags_mf (sum) |
| 04 | IP_flags_rb (sum) | 
| 05 | IP_frag_off (sum) |
| 06 | protocols (mean) |
| 07 | TCP_length (mean) |
| 08 | TCP_flags_ack (sum) |
| 09 | TCP_flags_cwr (sum) |
| 10 | TCP_flags_ece (sum) |
| 11 | TCP_flags_fin (sum) |
| 12 | TCP_flags_push (sum) |
| 13 | TCP_flags_res (sum) |
| 14 | TCP_flags_reset (sum) |
| 15 | TCP_flags_syn (sum) |
| 16 | TCP_flags_urg (sum) |
| 17 | TCP_window_size (mean) |
| 18 | UDP_length (mean) |
| 19 | ICMP_type (mean) |
| 20 | Packets (counter)|

In [17]:
# Author: Roberto Doriguzzi-Corin
# Project: Course on Network Intrusion and Anomaly Detection with Machine Learning
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import numpy as np
import glob
import h5py
import sys
import copy
import argparse
from sklearn.metrics import classification_report, mean_squared_error
import logging
import seaborn as sns
from util_functions import *
from IPython.display import Image, display
from sklearn.tree import export_graphviz
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

OUTPUT_FILE = "./rf_tree"
BENIGN_FOLDER = "./DOS2019/benign"
ATTACK_DNS = "./DOS2019/dns"
ATTACK_WEBDDOS = "./DOS2019/webddos"

from matplotlib import pyplot as plt
plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})

SEED=1
feature_names = get_feature_names()
target_names = ['benign', 'dns',  'syn', 'udplag', 'webddos'] #IMPORTANT: when adding new classes, maintain the alphabetical order
target_names_full = ['benign', 'dns', 'ldap', 'mssql', 'netbios', 'ntp', 'portmap', 'snmp', 'ssdp', 'syn', 'tftp', 'udp', 'udplag', 'webddos'] # we use this to match class names with the class numbers returned by the RF

X_train_benign, y_train_benign = load_dataset(BENIGN_FOLDER + "/*" + '-train.hdf5')
X_val_benign, y_val_benign = load_dataset(BENIGN_FOLDER + "/*" + '-val.hdf5')
X_test_dns, y_test_dns = load_dataset(ATTACK_DNS + "/*" + '-test.hdf5')
X_test_webddos, y_test_webddos = load_dataset(ATTACK_WEBDDOS + "/*" + '-test.hdf5')

X_test_attack = np.concatenate((X_test_dns,X_test_webddos))
y_test_attack = np.concatenate((y_test_dns,y_test_webddos))

# Dataset for regression problems
In the next cell, we modify the dataset to make it suitable for regression problems. In particular, we will use one of the features as target value (e.g, the average inter-arrival time of packets, or the average packet size). The selected feature is also removed from the samples.

In [18]:
Y_INDEX = 0 # index of the feature used as value to predict. 0 = averae IAT, 1 = average packet size of the flow

y_reg_train_benign = X_train_benign[:,Y_INDEX]
X_reg_train_benign = np.delete(X_train_benign, Y_INDEX, axis=1)

y_reg_val_benign = X_val_benign[:,Y_INDEX]
X_reg_val_benign = np.delete(X_val_benign, Y_INDEX, axis=1)

y_reg_test_attack = X_test_attack[:,Y_INDEX]
X_reg_test_attack = np.delete(X_test_attack, Y_INDEX, axis=1)

# Training the Linear Regressor
We train the Linear Regressor to predict the average packet size of the benign flows. 

In [ ]:
lr = LinearRegression()
lr.fit(X_reg_train_benign, y_reg_train_benign)

print("Bias: ", lr.intercept_) # bias
print("Weights: ", lr.coef_) # weights

# Validating the Linear Regressor with benign traffic
Now the Linear regressor is trained. We can use it to make predictions on unseen data. We first start with the validation set of benign traffic, which allows us to understand what is the average error of the prediction on it. 
This MSE and the standard deviation of the squared errors will be used to define the threshold for anomaly detection. We add the standard deviation to minimise false positives.

In [ ]:
y_reg_pred_benign = lr.predict(X_reg_val_benign)

benign_mse = mean_squared_error(y_reg_val_benign,y_reg_pred_benign)
print("MSE measures on the benign validation set: ", benign_mse)

benign_squared_errors = ((y_reg_val_benign-y_reg_pred_benign)**2)
print("Standard deviation of squared errors: ", benign_squared_errors.std())

# Define the threshold as the MSE+std_dev
attack_threshold = benign_mse + benign_squared_errors.std()
print ("Anomaly Threshold: ", attack_threshold)

# Plot the squared error distribution
plt.hist(benign_squared_errors, color = 'blue', edgecolor = 'black',bins=100)

# Testing the Linear Regressor with DDoS attack traffic
We test the Linear regressor on attack data samples. These samples have been generated by using DDoS attacks from the CIC-DDoS2019 dataset. We first measure the MSE and then we print the accuracy metrics obtained with the anomaly threshold computed in the previous step.

In [ ]:
y_reg_pred_attack = lr.predict(X_reg_test_attack)

attack_mse = mean_squared_error(y_reg_test_attack,y_reg_pred_attack)
print("MSE measures on the attack test set: ", attack_mse)

In [ ]:
attack_squared_errors = ((y_reg_test_attack-y_reg_pred_attack)**2)

# Plot the squared error distribution
plt.hist(attack_squared_errors, color = 'blue', edgecolor = 'black',bins=100)

# Accuracy metrics
We compute the accuracy metrics on the test sets of benign and attack traffic using the threshold computed above.

In [ ]:
import seaborn as sn
import pandas as pd

tn_fp = [True if err < attack_threshold else False for err in benign_squared_errors]
tp_fn = [True if err > attack_threshold else False for err in attack_squared_errors]

tn = sum(tn_fp)
tp = sum(tp_fn)
fp = len(tn_fp) - tn
fn = len(tp_fn) - tp

# Display the confusion matrix
conf_matrix = np.array([[tn,fp],[fn,tp]])

fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
accuracy = (tp+tn)/(tp+tn+fp+fn)
tpr = tp/(tp+fn) # recall
ppv = tp/(tp+fp) # precision
fpr = fp/(fp+tn) # false positive rate
fnr = fn/(fn+tp) # false negative rate
f1_score = 2*(tpr*ppv)/(tpr+ppv)


print ("Accuracy: ", accuracy)
print ("F1 Score: ", f1_score)
print ("FPR: ", fpr)
print ("FNR: ", fnr)

# Linear regression with polynomial features
You can use a linear model to fit nonlinear data. A simple way to do this is to add powers of each feature as new features, then train a linear model on this extended set of features. This technique is called Polynomial Regression. 

In [ ]:
# X_poly now contains the original feature of X plus the square of this feature
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly_train = poly_features.fit_transform(X_reg_train_benign)

lr = LinearRegression() 
lr.fit(X_poly_train, y_reg_train_benign)
print(lr.intercept_, lr.coef_) # print bias and weights

# Validating the Linear Regressor with polynomial features using the benign traffic
We can use the new regressor to make predictions on the validation set. Also in this case, we use the results of this test to set the anomaly threshold

In [ ]:
X_poly_val = poly_features.fit_transform(X_reg_val_benign)
y_reg_pred_benign = lr.predict(X_poly_val)

benign_mse = mean_squared_error(y_reg_val_benign,y_reg_pred_benign)
print("MSE measured on the benign validation set: ", benign_mse)

benign_squared_errors = ((y_reg_val_benign-y_reg_pred_benign)**2)
print("Standard deviation of squared errors: ", benign_squared_errors.std())

# Define the threshold as the MSE+std_dev
attack_threshold = benign_mse + benign_squared_errors.std()
print ("Anomaly Threshold: ", attack_threshold)

# Testing the Linear Regressor with DDoS attack traffic
We test the Linear regressor on attack data samples. These samples have been generated by using DDoS attacks from the CIC-DDoS2019 dataset. We first measure the MSE and then we print the accuracy metrics obtained with the anomaly threshold computed in the previous step.

In [ ]:
X_poly_test_attack = poly_features.fit_transform(X_reg_test_attack)
y_reg_pred_attack = lr.predict(X_poly_test_attack)
print (y_reg_pred_attack)

attack_mse = mean_squared_error(y_reg_test_attack,y_reg_pred_attack)
print("MSE measured on the benign test set: ", attack_mse)

# Accuracy metrics
We compute the accuracy metrics on the test sets of benign and attack traffic using the threshold computed above.

In [ ]:
import seaborn as sn
import pandas as pd

tn_fp = [True if err < attack_threshold else False for err in benign_squared_errors]
tp_fn = [True if err > attack_threshold else False for err in attack_squared_errors]

tn = sum(tn_fp)
tp = sum(tp_fn)
fp = len(tn_fp) - tn
fn = len(tp_fn) - tp

# Display the confusion matrix
conf_matrix = np.array([[tn,fp],[fn,tp]])

fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
accuracy = (tp+tn)/(tp+tn+fp+fn)
tpr = tp/(tp+fn) # recall
ppv = tp/(tp+fp) # precision
fpr = fp/(fp+tn) # false positive rate
fnr = fn/(fn+tp) # false negative rate
f1_score = 2*(tpr*ppv)/(tpr+ppv)


print ("Accuracy: ", accuracy)
print ("F1 Score: ", f1_score)
print ("FPR: ", fpr)
print ("FNR: ", fnr)